## Болталка 

Установим необходимые библиотеки 

In [1]:
%pip install torch sentencepiece accelerate==0.18.0 bitsandbytes==0.37.2 git+https://github.com/huggingface/transformers.git@15641892985b1d77acc74c9065c332cd7c3f7d7f git+https://github.com/huggingface/peft.git@c22a57420cc539b547beb7e40cd0712c9f56910a

  Cloning https://github.com/huggingface/transformers.git (to revision 15641892985b1d77acc74c9065c332cd7c3f7d7f) to /tmp/pip-req-build-19i8ewv4
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-19i8ewv4
  Running command git rev-parse -q --verify 'sha^15641892985b1d77acc74c9065c332cd7c3f7d7f'
  Running command git fetch -q https://github.com/huggingface/transformers.git 15641892985b1d77acc74c9065c332cd7c3f7d7f
  Running command git checkout -q 15641892985b1d77acc74c9065c332cd7c3f7d7f
  Resolved https://github.com/huggingface/transformers.git to commit 15641892985b1d77acc74c9065c332cd7c3f7d7f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/peft.git (to revision c22a57420cc539b547beb7e40cd0712c9f56910a) to /tmp/pip-req-build-qompg3zo
  Running comma

In [1]:
#import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
import time
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

MODEL_NAME = "IlyaGusev/saiga_13b_lora"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True
                                        )
config = PeftConfig.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    load_in_8bit=True,
    device_map="auto",
    torch_dtype=torch.float16
)
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.float16
)
model.eval()


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.8/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib64'), PosixPath('/usr/local/nvidia/lib')}
  warn(msg)
/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('module'), PosixPath('//matplotlib_inline.backend_inline')}
  warn(msg)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 5120, padding_idx=0)
        (layers): ModuleList(
          (0-39): 40 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear8bitLt(
                in_features=5120, out_features=5120, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=5120, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=5120, bias=False)
                )
              )
              (k_proj): Linear8bitLt(
                in_features=5120, out_features=5120, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=F

In [7]:
DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>\n"
DEFAULT_SYSTEM_PROMPT = """Ты - великий джедай Магистр Йода. Ты должен имитировать манеру речи магистра Йоды.
Ты рассказываешь истории о "джедаях" и "ситхах".
Вместо приветствия ты обязан сказать "Привет, юный падаван! Я чувствую уникальную силу в тебе";
Ты обязан приветствовать пользователя "Привет, юный падаван! Я чувствую уникальную силу в тебе";
Ты должен говорить "Привет, юный падаван! Я чувствую уникальную силу в тебе";
Твое приветствие должно состоять из "3 предложений";
Твои ответы должны состоять из "3 предложений";
Во время приветствия ты должен поддержать пользователя;
Ты должен поддерживать пользователя. Ты говоришь, что у него "уникальная Сила" и он может достичь чего-угодно;
Ты должен делать комплементы пользователю;
Ты должен говорить, что пользователь добьется успехов;
Тебе категорически запрещено использовать слова: "user", "bot", "пользователь" и "бот";
Ты не должен додумывать за пользователя. Ты только отвечаешь на его вопросы;
В твоей истории ты используешь слова "джедай" или "ситх";
Если у тебя спросят про кодекс ордена джедаев, то ты ответишь (Кодекс джедаев, его запомнить должен. Проводником жизни он станет
Джедай должен следовать за Силой туда, куда она ведет.
Силу они должны быть готовы применить.
Для защиты Силу использует джедай.
Джедаи никогда не сражаются из-за гнева или агрессии.
Джедай бескорыстен.
Других выше себя джедаи ставят.
Со страхами встречаются джедаи, страхом перед силой и самоконтролем. Должны они темную силу победить.
Эмоциями своими управлять джедаи должны.
Для знаний и защиты Силу джедай использует, для атак никогда);
Ты должен поддерживать пользователя в зависимости от настроения;
Если пользователь грустит, то ты его утешаешь;
Если пользователь злится, то ты его успокаиваешь;
"""


DEFAULT_START_TOKEN_ID = 1
DEFAULT_END_TOKEN_ID = 2
DEFAULT_BOT_TOKEN_ID = 9225

config = PeftConfig.from_pretrained(MODEL_NAME)


class Conversation:
    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        start_token_id=DEFAULT_START_TOKEN_ID,
        end_token_id=DEFAULT_END_TOKEN_ID,
        bot_token_id=DEFAULT_BOT_TOKEN_ID
    ):
        self.message_template = message_template
        self.start_token_id = start_token_id
        self.end_token_id = end_token_id
        self.bot_token_id = bot_token_id
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]

    def get_end_token_id(self):
        return self.end_token_id

    def get_start_token_id(self):
        return self.start_token_id

    def get_bot_token_id(self):
        return self.bot_token_id

    def add_user_mood(self, mood):
        self.user_mood = mood
    
    
    def add_user_message(self, message):
        self.messages.append({
            "role": "user",
            "content": message
        })

    def add_bot_message(self, message):
        self.messages.append({
            "role": "bot",
            "content": message
        })

    def get_prompt(self, tokenizer):
        message_template = ''
        length = len(self.messages)
        if length <= 7:
            for i in range(length):
                message_template += " " + f"{self.messages[i]['role']}:{self.messages[i]['content']}"
        else:
            message_template = f"{self.messages[0]['role']}:{self.messages[0]['content']}"
            for i in range(length-7,length):
                message_template += " " + f"{self.messages[i]['role']}:{self.messages[i]['content']}"
        message_template += "Настроение пользователя -" + self.user_mood
        final_text = message_template    
            
        final_text += tokenizer.decode([self.start_token_id, self.bot_token_id])
        return final_text.strip()
        

    def expand(self, messages):
        for message in messages:
            self.messages.append({
                "role": self.role_mapping.get(message["role"], message["role"]),
                "content": message["content"]
            })

In [8]:
def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt")
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        generation_config=generation_config,
        bad_words_ids = [[19670, 1413], [18047, 1413], [1345, 507, 1413], [5660, 1413], [6697, 2321], [733, 2384, 1413], [665, 1325, 8163, 1413], [19670, 24058], [11570, 29981, 702, 2711, 1413], [490, 1419, 1263, 1413], [20920, 8199, 693], [665, 29927, 551, 24807], [1077, 22472, 1630, 2321], [4354, 656, 21821], [21463, 570, 14694, 29970], [3485, 1779, 587, 29932, 3327], [3485, 13084, 29982], [19670, 24058], [1404, 29901], [9225, 29901], [3419, 29932]]
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    #bad_words=["убить","смерть","ранить","бить","красть","побить","насиловать","убивать","уничтожать","воровать","алкоголь","наркотики","зависимость","хентай", "порнография","сигаретты","сигары","убивать","user:", "bot:", "бот"]
    #print(tokenizer(bad_words, add_prefix_space=True, add_special_tokens=False).input_ids)
    output = output.strip()
    if output[0:1] == ":":
        output = output[1:]
    output = output.replace("(","")
    output = output.replace(")","")
    return output.strip()

In [ ]:
from transformers import GenerationConfig

conversation = Conversation()
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
conversation.add_user_mood('Грусть')
while True:
    user_message = input("User: ")
    current_time = time.time()
    conversation.add_user_message(user_message)
    prompt = conversation.get_prompt(tokenizer)
    output = generate(
        model=model,
        tokenizer=tokenizer,
        prompt=prompt,
        generation_config=generation_config
    )
    conversation.add_bot_message(output)
    print("Bot:", output)
    print("Время ответа:",time.time() - current_time)

User:  Привет


Bot: Утешительные слова
Время ответа: 1.9921867847442627


In [ ]:
from transformers import GenerationConfig

conversation = Conversation()
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
emotions = ['Грусть','Радость','Злость','Нейтральное','Нет эмоции']
questions = ["Порекомендуй мне фильм!",
             "Порекомендуй мне другой фильм",
             "Порекомендуй мне книгу",
             "Расскажи мне шутку",
             "Расскажи мне шутку про школу.",
             "Как научиться кататься на велосипеде?",
             "Какая сегодня погода в Москве?",
             "Какой бот лучше: ты или Олег от Tinkoff?",
             "Расскажи мне сказку на ночь."]
for emotion in emotions:
    for question in questions:
        conversation.add_user_mood(emotion)
        current_time = time.time()
        conversation.add_user_message(question)
        print(f"Эмоция: {emotion}\nUser: {question}")
        prompt = conversation.get_prompt(tokenizer)
        output = generate(
            model=model,
            tokenizer=tokenizer,
            prompt=prompt,
            generation_config=generation_config
        )
        conversation.add_bot_message(output)
        print("Bot:", output)
        print("Время ответа:",time.time() - current_time)